# Pre Processing
**In this file, we will analyze the initial dataset consisting of CSV files in order to create the final dataset. The final dataset, composed of the relevant columns, will be used to train the predictive models**

Imported libraries

In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

Creating the pyspark session

In [2]:
# Create the session
conf = SparkConf(). \
    set('spark.ui.port', "4050"). \
    set('spark.executor.memory', '15G'). \
    set('spark.driver.memory', '50G'). \
    set('spark.driver.maxResultSize', '40G'). \
    setAppName("PySparkProject"). \
    set('spark.executor.cores', "10"). \
    setMaster("local[*]")

sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()

sc._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.memory', '15G'),
 ('spark.app.submitTime', '1683842941574'),
 ('spark.driver.host', 'MarcoPC.homenet.telecomitalia.it'),
 ('spark.app.id', 'local-1683842942929'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.me

**Work on dataset begins**

Loading dataframes

In [3]:
# load the cvs files into dataframes
path = "C:/Users/marco/Desktop"

try:
    df_appearances = spark.read.csv("archive/appearances.csv", header=True, inferSchema=True)
    df_club_games = spark.read.csv("archive/club_games.csv", header=True, inferSchema=True)
    df_clubs = spark.read.csv("archive/clubs.csv", header=True, inferSchema=True)
    df_competitions = spark.read.csv("archive/competitions.csv", header=True, inferSchema=True)
    df_game_events = spark.read.csv("archive/game_events.csv", header=True, inferSchema=True)
    df_games = spark.read.csv("archive/games.csv", header=True, inferSchema=True)
    df_player_valuations = spark.read.csv("archive/player_valuations.csv", header=True, inferSchema=True)
    df_players = spark.read.csv("archive/players.csv", header=True, inferSchema=True)
except Exception:
    df_appearances = spark.read.csv(path+"/archive/appearances.csv", header=True, inferSchema=True)
    df_club_games = spark.read.csv(path+"/archive/club_games.csv", header=True, inferSchema=True)
    df_clubs = spark.read.csv(path+"/archive/clubs.csv", header=True, inferSchema=True)
    df_competitions = spark.read.csv(path+"/archive/competitions.csv", header=True, inferSchema=True)
    df_game_events = spark.read.csv(path+"/archive/game_events.csv", header=True, inferSchema=True)
    df_games = spark.read.csv(path+"/archive/games.csv", header=True, inferSchema=True)
    df_player_valuations = spark.read.csv(path+"/archive/player_valuations.csv", header=True, inferSchema=True)
    df_players = spark.read.csv(path+"/archive/players.csv", header=True, inferSchema=True)

In [4]:
# drop duplicates df_club_games
df_club_games = df_club_games.dropDuplicates(['club_id', 'own_goals', 'own_position', 'own_manager_name', 'opponent_id', 'opponent_goals', 'opponent_position', 'opponent_manager_name', 'hosting', 'is_win'])

In [5]:
# drop duplicates df_games
df_games = df_games.dropDuplicates(['home_club_id', 'away_club_id', 'date'])

In [6]:
# delete from df_appareances the instances where player_club_id is not in the df_clubs dataframe (club ids)
# beacuse in df_appereances there are some player_club_id that do not exist
df_appearances = df_appearances.join(df_clubs, df_appearances.player_club_id == df_clubs.club_id, 'inner').select(df_appearances.columns)

In [7]:
# join players and appearances
df_players_appearances = df_players.join(df_appearances, ["player_id"], how='inner')

In [8]:
 # join players_appearances and club_games to extract information about the games played by the player
#rinomina player_club_id in club_id
df_players_appearances = df_players_appearances.withColumnRenamed("player_club_id", "club_id")
df_players_appearances = df_players_appearances.join(df_club_games, ["game_id", "club_id"], how='inner')

In [9]:
# drop useless and duplicated features from df_players_appearances
df_players_appearances = df_players_appearances.drop("current_club_id", "appearance_id",
                                                        "highest_market_value_in_eur", "current_club_name",
                                                        "city_of_birth", "market_value_in_eur",
                                                        "contract_expiration_date", "agent_name",
                                                        "current_club_domestic_competition_id", "image_url",
                                                        "last_season", "url", "player_current_club_id",
                                                        "first_name", "last_name", "player_name", "player_code")

In [10]:
# drop useless and duplicated features from df_players_valuations
df_player_valuations = df_player_valuations.drop("datetime", "dateweek")

In [11]:
# rename the column date of df_players_valuations in date_v to avoid confusion with the date of df_players_appearances
df_player_valuations = df_player_valuations.withColumnRenamed("date", "date_v")

In [12]:
# Join the two dataframes on player_id
df = df_player_valuations.join(df_players_appearances, "player_id")

In [13]:
# TODO decide if we want to keep the players with no appearances, in case we have to do a union with valuations
# adding before the zeroed column of df_players_appearances

In [14]:
# we want to keep only the rows where the appearence date is within 1 year from the valuation date
df = df.filter(
    (year(df.date_v) == year(df.date) + 1) & (month(df.date_v) < month(df.date)) |
    (year(df.date_v) == year(df.date)) & (month(df.date_v) > month(df.date)) |
    (year(df.date_v) == year(df.date)) & (month(df.date_v) == month(df.date)) & (dayofmonth(df.date_v) > dayofmonth(df.date)) |
    (year(df.date_v) == year(df.date) + 1) & (month(df.date_v) == month(df.date)) & (dayofmonth(df.date_v) < dayofmonth(df.date))
).dropDuplicates(["player_id", "date", "date_v"])

We add the statistics of the teams in which the player played during the year and in which the player took the field (appearance)

In [15]:
# add the is_draw to count the draws
df = df.withColumn("is_draw", when(df.own_goals == df.opponent_goals, 1).otherwise(0))

# add column games_played that counts the instances with key (player_id, date_v)
df = df.withColumn("appearances", count(df.date_v).over(Window.partitionBy("date_v", "player_id")))

# add column games_won that counts the instances with key (player_id, date_v) and is_win = 1
df = df.withColumn("games_won_pl",
                                count(when(df.own_goals > df.opponent_goals, 1)).over(Window.partitionBy("date_v", "player_id")))

# add column games_draw_pl that counts the instances with key (player_id, date_v) and is_draw = 1
df = df.withColumn("games_draw_pl",
                                count(when(df.is_draw == 1, 1)).over(Window.partitionBy("date_v", "player_id")))

df = df.withColumn("games_lost_pl", count(when(df.own_goals < df.opponent_goals, 1)).over(Window.partitionBy("date_v", "player_id")))

In [16]:
# delete some columns
df = df.drop("game_id", "own_goals", "own_position", "own_manager_name", "opponent_id",
                        "opponent_goals", "opponent_position", "opponent_manager_name", "hosting", "is_win",
                        "is_draw")

# for each (player_id, date_v) add a column with the result of (games_won_pl*3 + games_draw_pl)/games_played
df = df.withColumn("winning_rate_pl",
                                    (df.games_won_pl * 3 + df.games_draw_pl) / df.appearances)

In [17]:
# Group by the player_id and the valuation date and extract all the important features
df = df.groupBy(
    "player_id", "name", col("market_value_in_eur").alias("market_value"), "date_v",
    col("current_club_id").alias("current_club_id"), col("height_in_cm").alias("height"),
    col("country_of_citizenship").alias("citizenship"), col("date_of_birth").alias("date_birth"), "position",
    "sub_position", "appearances", "games_won_pl", "games_draw_pl", "games_lost_pl", "winning_rate_pl") \
    .agg(collect_set("competition_id").alias("competition_id"),
            collect_set("club_id").alias("club_id"),
            sum("assists").alias("assists"),
            count("date_v").alias("appearances2"),
            sum("goals").alias("goals"),
            sum("minutes_played").alias("minutes_played"),
            sum("red_cards").alias("red_cards"),
            sum("yellow_cards").alias("yellow_cards"))

#df.filter(df.player_id == 148455).show()

In [18]:
#add last valuation in temporal terms
df = df.withColumn("last_valuation", lag(df.market_value).over(Window.partitionBy("player_id").orderBy("date_v")))

We add the statistics of the teams in which the player played during the year

In [19]:
# join club_games and games to extract the date from games
df_club_games_join = df_club_games.join(df_games.select("game_id", "date"), "game_id", how='inner')

In [20]:
# Expands the player_club_id list into separate columns
df_result_expanded = df.withColumn("club_id", explode(col("club_id")))

In [21]:
# Peform join based on club_id and apply condition on date
df = df_result_expanded.join(df_club_games_join, ["club_id"]) \
    .where(expr("date <= date_v AND date >= date_v - INTERVAL 1 YEAR"))

In [22]:
# Select columns
df = df.select(df.columns + df_club_games_join.columns)

In [23]:
# add column draw that is equal to 1 if own_goals = opponent_goals
df = df.withColumn("is_draw", when(df.own_goals == df.opponent_goals, 1).otherwise(0))

#add column games_played_club that counts the instances with key (player_id, date_v)
df = df.withColumn("games_played_club", count(df.date_v).over(Window.partitionBy("date_v", "player_id")))

# add column games_won_club that counts the instances with key (player_id, date_v) and is_win = 1
df = df.withColumn("games_won_club", count(when(df.own_goals > df.opponent_goals, 1)).over(Window.partitionBy("date_v", "player_id")))

# add column games_draw_club that counts the instances with key (player_id, date_v) and is_draw = 1
df = df.withColumn("games_draw_club", count(when(df.is_draw == 1, 1)).over(Window.partitionBy("date_v", "player_id")))

# add column games_lost_club that counts the instances with key (player_id, date_v) and is_win = 0 and is_draw = 0
df = df.withColumn("games_lost_club", count(when(df.own_goals < df.opponent_goals, 1)).over(Window.partitionBy("date_v", "player_id")))

In [24]:
#delete some columns
df = df.drop("game_id", "own_goals",  "own_position", "own_manager_name", "opponent_id", "opponent_goals", "opponent_position", "opponent_manager_name", "hosting", "is_win", "date", "is_draw")

In [25]:
#df = df.dropDuplicates(["player_id", "date_v"])
df = df.dropDuplicates()

In [26]:
# for each (player_id, date_v) add a column with the result of (games_won_club*3 + games_draw_club)/games_played_club
df = df.withColumn("winning_rate_club", (df.games_won_club*3 + df.games_draw_club)/df.games_played_club)

In [27]:
#arrange columns, delete and rename some columns, aggregate club_id
df = df.groupBy(
    "player_id", "date_v", "market_value", "name", "date_birth", "current_club_id", "height", "citizenship", "position", "sub_position", col("competition_id").alias("competitions_id"),
    "assists", "goals", "minutes_played", "red_cards", "yellow_cards", "last_valuation", "appearances", "games_won_pl", "games_draw_pl", "games_lost_pl", "winning_rate_pl",
    "games_played_club", "games_won_club", "games_draw_club", "games_lost_club", "winning_rate_club") \
    .agg(collect_set("club_id").alias("clubs_id"))

In [28]:
# add column age that is equal to date_v - date_birth cifra intera
df = df.withColumn("age", floor(datediff(df.date_v, df.date_birth)/365))

In [29]:
# take only the first digit after the comma of winning_rate_pl and winning_rate_club
df = df.withColumn("winning_rate_pl", round(df.winning_rate_pl, 1))
df = df.withColumn("winning_rate_club", round(df.winning_rate_club, 1))

In [30]:
df = df.select("player_id", "date_v", "market_value", "name", "date_birth", "age", "current_club_id", "height", "citizenship", "position", "sub_position", "competitions_id", "clubs_id",
    "assists", "goals", "minutes_played", "red_cards", "yellow_cards", "last_valuation", "appearances", "games_won_pl", "games_draw_pl", "games_lost_pl", "winning_rate_pl",
    "games_played_club", "games_won_club", "games_draw_club", "games_lost_club", "winning_rate_club")

In [31]:
df.filter(df.player_id == 148455).show(50)

+---------+-------------------+------------+-------------+-------------------+---+---------------+------+-----------+--------+------------+--------------------+--------------+-------+-----+--------------+---------+------------+--------------+-----------+------------+-------------+-------------+---------------+-----------------+--------------+---------------+---------------+-----------------+
|player_id|             date_v|market_value|         name|         date_birth|age|current_club_id|height|citizenship|position|sub_position|     competitions_id|      clubs_id|assists|goals|minutes_played|red_cards|yellow_cards|last_valuation|appearances|games_won_pl|games_draw_pl|games_lost_pl|winning_rate_pl|games_played_club|games_won_club|games_draw_club|games_lost_club|winning_rate_club|
+---------+-------------------+------------+-------------+-------------------+---+---------------+------+-----------+--------+------------+--------------------+--------------+-------+-----+--------------+------

In [32]:
# TODO trattare null values

In [33]:
import pandas as pd
df_pandas = df.toPandas()

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


We transform the column competitions id and clubs id, that are array of strings in a single binary value using label binarization.

In [ ]:
# apply pd.get_dummies to the column of arrays
dummies = pd.get_dummies(df_pandas["competitions_id"].apply(pd.Series).stack()).sum(level=0)
# concatenate the dummy variables into a single string
dummies["comp_string"] = dummies.apply(lambda x: "".join(x.astype(str)), axis=1)
# join the dummies dataframe with the original dataframe
df_pandas = df_pandas.join(dummies["comp_string"])
#df_pandas = df_pandas.drop("competitions_id")


In [35]:
# apply pd.get_dummies to the column of arrays
dummies = pd.get_dummies(df_pandas["clubs_id"].apply(pd.Series).stack()).sum(level=0)
# concatenate the dummy variables into a single string
dummies["club_str"] = dummies.apply(lambda x: "".join(x.astype(str)), axis=1)
# join the dummies dataframe with the original dataframe
df_pandas = df_pandas.join(dummies["club_str"])
#df_pandas = df_pandas.drop("club_str")

C:\Users\marco\AppData\Local\Temp\ipykernel_15696\1431364872.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_pandas["clubs_id"].apply(pd.Series).stack()).sum(level=0)


In [36]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135917 entries, 0 to 135916
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   player_id          135917 non-null  int32         
 1   date_v             135917 non-null  datetime64[ns]
 2   market_value       135917 non-null  int32         
 3   name               135917 non-null  object        
 4   date_birth         135878 non-null  datetime64[ns]
 5   age                135878 non-null  float64       
 6   current_club_id    135917 non-null  int32         
 7   height             135917 non-null  int32         
 8   citizenship        135917 non-null  object        
 9   position           135917 non-null  object        
 10  sub_position       124406 non-null  object        
 11  competitions_id    135917 non-null  object        
 12  clubs_id           135917 non-null  object        
 13  assists            135917 non-null  int64   

In [40]:
df_pandas

,player_id,date_v,market_value,name,date_birth,age,current_club_id,height,citizenship,position,...,games_draw_pl,games_lost_pl,winning_rate_pl,games_played_club,games_won_club,games_draw_club,games_lost_club,winning_rate_club,comp_string,club_str
0,26,2015-02-04,3000000,Roman Weidenfeller,1980-08-06,34.0,16,190,Germany,Goalkeeper,...,3,8,1.3,50,26,7,17,1.7,0000010000000000000000001000000000000000000,0000000001000000000000000000000000000000000000...
1,26,2015-07-01,2000000,Roman Weidenfeller,1980-08-06,34.0,16,190,Germany,Goalkeeper,...,5,13,1.5,49,23,8,18,1.6,0000010000000000000000001000000000000000000,0000000001000000000000000000000000000000000000...
2,26,2015-10-16,1000000,Roman Weidenfeller,1980-08-06,35.0,16,190,Germany,Goalkeeper,...,5,10,1.6,54,29,10,15,1.8,0000010000001100000000001000000000000000000,0000000001000000000000000000000000000000000000...
3,26,2016-02-15,1000000,Roman Weidenfeller,1980-08-06,35.0,16,190,Germany,Goalkeeper,...,3,6,1.8,54,36,7,11,2.1,0000010000001100000000001000000000000000000,0000000001000000000000000000000000000000000000...
4,26,2016-07-22,1000000,Roman Weidenfeller,1980-08-06,35.0,16,190,Germany,Goalkeeper,...,2,3,2.1,56,40,8,8,2.3,0000000000001100000000001000000000000000000,0000000001000000000000000000000000000000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135912,987700,2022-11-21,300000,Jonathan Okoronkwo,2003-09-13,19.0,16704,175,Nigeria,Attack,...,1,1,2.0,36,16,9,11,1.6,0000000000000000000000000000000110000000000,0000000000000000000000000000000000000000000000...
135913,1014243,2023-02-01,400000,Vivaldo,2005-01-28,18.0,410,192,Portugal,Attack,...,1,0,1.0,39,15,13,11,1.5,0000000000000000000000100000000000000000000,0000000000000000000000000000000000000000000000...
135914,1028566,2022-10-27,50000,Mert Kurt,2002-10-25,20.0,11688,180,Turkey,Attack,...,0,3,0.0,38,12,9,17,1.2,0000000000000000000000000000000000000010000,0000000000000000000000000000000000000000000000...
135915,1028566,2023-03-14,75000,Mert Kurt,2002-10-25,20.0,11688,180,Turkey,Attack,...,0,4,0.0,34,8,12,14,1.1,0000000000000000000000000000000000000010000,0000000000000000000000000000000000000000000000...


In [41]:
#write the dataframe in csv format
df_pandas.to_csv('data.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
#from autoviz.AutoViz_Class import AutoViz_Class
#AV = AutoViz_Class()
#df = AV.AutoViz('data.csv')